In [1]:
from IPython.display import display, Markdown, Latex
import pandas as pd
import numpy as np
import json
import os

## Experiments and Functions

In [27]:
experiments = [
            #{'id': '1', 'source':'imdb', 'target':'uwcse', 'predicate':'workedunder', 'to_predicate':'advisedby', 'arity': 2},
            #{'id': '2', 'source':'uwcse', 'target':'imdb', 'predicate':'advisedby', 'to_predicate':'workedunder', 'arity': 2},
            #{'id': '3', 'source':'imdb', 'target':'cora', 'predicate':'workedunder', 'to_predicate':'samevenue', 'arity': 2},
            #{'id': '4', 'source':'cora', 'target':'imdb', 'predicate':'samevenue', 'to_predicate':'workedunder', 'arity': 2},
            ##{'id': '5', 'source':'cora', 'target':'imdb', 'predicate':'sametitle', 'to_predicate':'workedunder', 'arity': 2},
            ##{'id': '6', 'source':'imdb', 'target':'cora', 'predicate':'workedunder', 'to_predicate':'sametitle', 'arity': 2},
            ##{'id': '5', 'source':'uwcse', 'target':'cora', 'predicate':'advisedby', 'to_predicate':'samevenue', 'arity': 2},
            ##{'id': '6', 'source':'cora', 'target':'uwcse', 'predicate':'samevenue', 'to_predicate':'advisedby', 'arity': 2},
            #{'id': '7', 'source':'yeast', 'target':'twitter', 'predicate':'proteinclass', 'to_predicate':'accounttype', 'arity': 2},
            #{'id': '8', 'source':'twitter', 'target':'yeast', 'predicate':'accounttype', 'to_predicate':'proteinclass', 'arity': 2},
            {'id': '9', 'source':'nell_sports', 'target':'nell_finances', 'predicate':'teamplayssport', 'to_predicate':'companyeconomicsector', 'arity': 2},
            {'id': '10', 'source':'nell_finances', 'target':'nell_sports', 'predicate':'companyeconomicsector', 'to_predicate':'teamplayssport', 'arity': 2},
]

In [3]:
def load_data(experiments, metric, directory):
    data = {}
    for item in experiments:
        experiment_title = item['id'] + '_' + item['source'] + '_' + item['target']
        
        if(metric != ''):
            file_path = os.getcwd() + directory + experiment_title + '/' + experiment_title + '_fasttext_{}.json'.format(metric)
        else:
            file_path = os.getcwd() + directory + experiment_title + '/' + experiment_title + '.json'
            
        #if(os.path.isfile(file_path)):
        with open(file_path, 'r') as fp:
            results = json.load(fp)

            data[experiment_title] = results
    return data

## Estudo dos Mapeamentos

### Depth-First Mapping

In [30]:
# Setting paths parameters
mapping_order = 'deep'
k = 1
compare_stopwords = False

#Removing Yeast and Twitter pair of experiments because there is no stopwords in predicates
#exp = experiments.copy()
#exp.remove({'id': '7', 'source':'yeast', 'target':'twitter', 'predicate':'proteinclass', 'to_predicate':'accounttype', 'arity': 2})
#exp.remove({'id': '8', 'source':'twitter', 'target':'yeast', 'predicate':'accounttype', 'to_predicate':'proteinclass', 'arity': 2})

In [31]:
# Using stopwords
path = f'/results/transfer/{mapping_order}/w_stopwords/k_{k}/'
data_softcosine_stopwords = load_data(experiments, 'softcosine', path)
data_euclidean_stopwords = load_data(experiments, 'euclidean', path)
data_wmd_stopwords = load_data(experiments, 'wmd', path)
data_rwmd_stopwords = load_data(experiments, 'relax-wmd', path)

if compare_stopwords:
    # Removing stopwords
    path = f'/results/transfer/{mapping_order}/no_stopwords/k_{k}/'
    data_softcosine_no_stopwords = load_data(experiments, 'softcosine', path)
    data_euclidean_no_stopwords = load_data(experiments, 'euclidean', path)
    data_wmd_no_stopwords = load_data(experiments, 'wmd', path)
    data_rwmd_no_stopwords = load_data(experiments, 'relax-wmd', path)

baseline_path = f'/results/baselines/transfer/'
baseline_data = load_data(experiments, '', baseline_path)

for metric in ['CLL', 'AUC ROC', 'AUC PR', 'Learning time']: #, 'Recall', 'F1', 'Precision', 'Learning and Revision time', 'Inference time']:
    display(Markdown('# Results for ' + metric))
    table = []
    for j in range(len(experiments)):
            
        dataset = experiments[j]['id'] + '_' + experiments[j]['source'] + '_' + experiments[j]['target']
        if dataset in data_euclidean_stopwords:
            mapping_time = ''
            results_using_stopwords = {}
            results_removing_stopwords = {}

            # Results using Stopwords
            results_using_stopwords['softcosine'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_softcosine_stopwords[str(dataset)]])
            results_using_stopwords['euclidean'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_euclidean_stopwords[str(dataset)]])
            results_using_stopwords['wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_wmd_stopwords[str(dataset)]])
            results_using_stopwords['relax-wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_rwmd_stopwords[str(dataset)]])

            #results_using_stopwords['softcosine no revision'] = np.array([(np.array([item['transfer']['parameter'][metric] for item in sublist])).mean() for sublist in data_softcosine_stopwords[str(dataset)]])
            #results_using_stopwords['euclidean no revision'] = np.array([(np.array([item['transfer']['parameter'][metric] for item in sublist])).mean() for sublist in data_euclidean_stopwords[str(dataset)]])
            #results_using_stopwords['wmd no revision'] = np.array([(np.array([item['transfer']['parameter'][metric] for item in sublist])).mean() for sublist in data_wmd_stopwords[str(dataset)]])
            #results_using_stopwords['relax-wmd no revision'] = np.array([(np.array([item['transfer']['parameter'][metric] for item in sublist])).mean() for sublist in data_rwmd_stopwords[str(dataset)]])

            if compare_stopwords:
                # Results removing stopwords
                results_removing_stopwords['softcosine'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_softcosine_no_stopwords[str(dataset)]])
                results_removing_stopwords['euclidean'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_euclidean_no_stopwords[str(dataset)]])
                results_removing_stopwords['wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_wmd_no_stopwords[str(dataset)]])
                results_removing_stopwords['relax-wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_rwmd_no_stopwords[str(dataset)]])

            if(experiments[j]['id'] != '10'):
                treeboostler = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in baseline_data[str(dataset)]])
                treeboostler_no_revision = np.array([(np.array([item['transfer']['parameter'][metric] for item in sublist])).mean() for sublist in baseline_data[str(dataset)]])
                #print(treeboostler)
            rdnb = np.array([(np.array([item['rdn_b'][metric] for item in sublist])).mean() for sublist in baseline_data[str(dataset)]])
            
            if(experiments[j]['id'] != '10'):
                table.append([dataset,
                '%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
                '%.3f +/- %.3f' % (treeboostler_no_revision.mean(), 2 * treeboostler_no_revision.std()),
                '%.3f +/- %.3f' % (results_using_stopwords['softcosine'].mean(), 2 * results_using_stopwords['softcosine'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['euclidean'].mean(), 2 * results_using_stopwords['euclidean'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['wmd'].mean(), 2 * results_using_stopwords['wmd'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['relax-wmd'].mean(), 2 * results_using_stopwords['relax-wmd'].std()),
                             ])
            else:
                table.append([dataset,
                '%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
                'NaN +/- NaN',
                '%.3f +/- %.3f' % (results_using_stopwords['softcosine'].mean(), 2 * results_using_stopwords['softcosine'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['euclidean'].mean(), 2 * results_using_stopwords['euclidean'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['wmd'].mean(), 2 * results_using_stopwords['wmd'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['relax-wmd'].mean(), 2 * results_using_stopwords['relax-wmd'].std()),
              ])
            
            #table.append(['', '', '', '', '', '', ''])
           
            if(experiments[j]['id'] != '10' and compare_stopwords):
                table.append([dataset + '*',
                '%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
                '%.3f +/- %.3f' % (treeboostler_no_revision.mean(), 2 * treeboostler_no_revision.std()),              
                '%.3f +/- %.3f' % (results_removing_stopwords['softcosine'].mean(), 2 * results_removing_stopwords['softcosine'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['euclidean'].mean(), 2 * results_removing_stopwords['euclidean'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['wmd'].mean(), 2 * results_removing_stopwords['wmd'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['relax-wmd'].mean(), 2 * results_removing_stopwords['relax-wmd'].std()),                
                             ])
            elif(compare_stopwords):
                table.append([dataset + '*',
                '%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
                'NaN +/- NaN',
                '%.3f +/- %.3f' % (results_removing_stopwords['softcosine'].mean(), 2 * results_removing_stopwords['softcosine'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['euclidean'].mean(), 2 * results_removing_stopwords['euclidean'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['wmd'].mean(), 2 * results_removing_stopwords['wmd'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['relax-wmd'].mean(), 2 * results_removing_stopwords['relax-wmd'].std()),
                             ])
            
            # No Revision
            #table.append([dataset + '_NR',
            #    '%.3f +/- %.3f' % (results_using_stopwords['softcosine no revision'].mean(), 2 * results_using_stopwords['softcosine'].std()),
            #    '%.3f +/- %.3f' % (results_using_stopwords['euclidean no revision'].mean(), 2 * results_using_stopwords['euclidean'].std()),
            #    '%.3f +/- %.3f' % (results_using_stopwords['wmd no revision'].mean(), 2 * results_using_stopwords['wmd'].std()),
            #    '%.3f +/- %.3f' % (results_using_stopwords['relax-wmd no revision'].mean(), 2 * results_using_stopwords['relax-wmd'].std()),
            #    '%.3f +/- %.3f' % (treeboostler_no_revision.mean(), 2 * treeboostler_no_revision.std()),
            #    '%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
            #])
            #table.append(['', '', '', '', '', '', ''])

    #display(pd.DataFrame(table, columns=['Experiment', 'RDN-B'])) #, 'Transfer Learning with Revision Theory - Relax WMD'])) #, 'Learning from scratch (RDN-B)']))
    display(pd.DataFrame(table, columns=['Experiment', 'RDN-B', 'TreeBoostler', 'Transfer Learning - SoftCosine', 'Transfer Learning - Euclidean', 'Transfer Learning - WMD', 'Transfer Learning - Relax-WMD'])) #, 'Learning from scratch (RDN-B)']))
    #display(pd.DataFrame(table, columns=['Experiment', 'Transfer Learning', 'Transfer Learning with Revision Theory']))

# Results for CLL

,Experiment,RDN-B,TreeBoostler,Transfer Learning - SoftCosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - Relax-WMD
0,9_nell_sports_nell_finances,-0.303 +/- 0.006,-0.314 +/- 0.012,-0.367 +/- 0.005,-0.368 +/- 0.004,-0.366 +/- 0.003,-0.367 +/- 0.002
1,10_nell_finances_nell_sports,-0.085 +/- 0.001,NaN +/- NaN,-0.371 +/- 0.008,-0.368 +/- 0.003,-0.374 +/- 0.007,-0.366 +/- 0.006


# Results for AUC ROC

,Experiment,RDN-B,TreeBoostler,Transfer Learning - SoftCosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - Relax-WMD
0,9_nell_sports_nell_finances,0.762 +/- 0.041,0.733 +/- 0.040,0.557 +/- 0.001,0.558 +/- 0.002,0.539 +/- 0.000,0.541 +/- 0.002
1,10_nell_finances_nell_sports,0.994 +/- 0.001,NaN +/- NaN,0.484 +/- 0.003,0.488 +/- 0.001,0.487 +/- 0.002,0.492 +/- 0.002


# Results for AUC PR

,Experiment,RDN-B,TreeBoostler,Transfer Learning - SoftCosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - Relax-WMD
0,9_nell_sports_nell_finances,0.089 +/- 0.017,0.090 +/- 0.016,0.002 +/- 0.000,0.002 +/- 0.000,0.002 +/- 0.000,0.002 +/- 0.000
1,10_nell_finances_nell_sports,0.313 +/- 0.047,NaN +/- NaN,0.002 +/- 0.000,0.002 +/- 0.000,0.002 +/- 0.000,0.002 +/- 0.000


# Results for Learning time

,Experiment,RDN-B,TreeBoostler,Transfer Learning - SoftCosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - Relax-WMD
0,9_nell_sports_nell_finances,42.075 +/- 9.081,29.710 +/- 15.034,8.192 +/- 0.302,7.014 +/- 0.216,7.588 +/- 0.085,46.281 +/- 0.119
1,10_nell_finances_nell_sports,349.809 +/- 11.872,NaN +/- NaN,27.099 +/- 0.259,28.003 +/- 0.544,28.400 +/- 0.264,57.413 +/- 0.182


### Ranked-First Mapping

In [24]:
# Setting paths parameters
mapping_order = 'most_similar'
k = '1'

compare_stopwords = False

In [26]:
# Using stopwords
path = f'/results/transfer/{mapping_order}/w_stopwords/k_{k}/'
data_softcosine_stopwords = load_data(experiments, 'softcosine', path)
data_euclidean_stopwords = load_data(experiments, 'euclidean', path)
data_wmd_stopwords = load_data(experiments, 'wmd', path)
data_rwmd_stopwords = load_data(experiments, 'relax-wmd', path)

if compare_stopwords:
    # Removing stopwords
    path = f'/results/transfer/{mapping_order}/no_stopwords/k_{k}/'
    data_softcosine_no_stopwords = load_data(experiments, 'softcosine', path)
    data_euclidean_no_stopwords = load_data(experiments, 'euclidean', path)
    data_wmd_no_stopwords = load_data(experiments, 'wmd', path)
    data_rwmd_no_stopwords = load_data(experiments, 'relax-wmd', path)

baseline_path = f'/results/baselines/transfer/'
baseline_data = load_data(experiments, '', baseline_path)

for metric in ['CLL', 'AUC ROC', 'AUC PR', 'Learning time']: #, 'Recall', 'F1', 'Precision', 'Learning and Revision time', 'Inference time']:
    display(Markdown('# Results for ' + metric))
    table = []
    
    for j in range(len(experiments)):
            
        dataset = experiments[j]['id'] + '_' + experiments[j]['source'] + '_' + experiments[j]['target']
        if dataset in data_euclidean_stopwords:
            mapping_time = ''
            results_using_stopwords = {}
            results_removing_stopwords = {}

            # Results using Stopwords
            results_using_stopwords['softcosine'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_softcosine_stopwords[str(dataset)]])
            results_using_stopwords['euclidean'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_euclidean_stopwords[str(dataset)]])
            results_using_stopwords['wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_wmd_stopwords[str(dataset)]])
            results_using_stopwords['relax-wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_rwmd_stopwords[str(dataset)]])
            
            if compare_stopwords:
                # Results removing stopwords
                results_removing_stopwords['softcosine'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_softcosine_no_stopwords[str(dataset)]])
                results_removing_stopwords['euclidean'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_euclidean_no_stopwords[str(dataset)]])
                results_removing_stopwords['wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_wmd_no_stopwords[str(dataset)]])
                results_removing_stopwords['relax-wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_rwmd_no_stopwords[str(dataset)]])
            
            if(experiments[j]['id'] != '10'):
                treeboostler = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in baseline_data[str(dataset)]])
                #treeboostler_no_revision = np.array([(np.array([item['transfer_{}'.format(amount)]['parameter'][metric] for item in sublist])).mean() for sublist in treeboostler_data[str(dataset)]])
                #print(treeboostler)
            rdnb = np.array([(np.array([item['rdn_b'][metric] for item in sublist])).mean() for sublist in baseline_data[str(dataset)]])
            
            if(experiments[j]['id'] != '10'):
                table.append([dataset,
                '%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
                '%.3f +/- %.3f' % (treeboostler.mean(), 2 * treeboostler.std()),
                '%.3f +/- %.3f' % (results_using_stopwords['softcosine'].mean(), 2 * results_using_stopwords['softcosine'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['euclidean'].mean(), 2 * results_using_stopwords['euclidean'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['wmd'].mean(), 2 * results_using_stopwords['wmd'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['relax-wmd'].mean(), 2 * results_using_stopwords['relax-wmd'].std()),
                             ])
            else:
                table.append([dataset,
                '%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
                'NaN +/- NaN',
                '%.3f +/- %.3f' % (results_using_stopwords['softcosine'].mean(), 2 * results_using_stopwords['softcosine'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['euclidean'].mean(), 2 * results_using_stopwords['euclidean'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['wmd'].mean(), 2 * results_using_stopwords['wmd'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['relax-wmd'].mean(), 2 * results_using_stopwords['relax-wmd'].std()),
              ])
            
            #table.append(['', '', '', '', '', '', ''])
           
            if(experiments[j]['id'] != '10' and compare_stopwords):
                table.append([dataset + '*',
                '%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
                '%.3f +/- %.3f' % (treeboostler.mean(), 2 * treeboostler.std()),              
                '%.3f +/- %.3f' % (results_removing_stopwords['softcosine'].mean(), 2 * results_removing_stopwords['softcosine'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['euclidean'].mean(), 2 * results_removing_stopwords['euclidean'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['wmd'].mean(), 2 * results_removing_stopwords['wmd'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['relax-wmd'].mean(), 2 * results_removing_stopwords['relax-wmd'].std()),                
                             ])
            elif(compare_stopwords):
                table.append([dataset + '*',
                '%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
                'NaN +/- NaN',
                '%.3f +/- %.3f' % (results_removing_stopwords['softcosine'].mean(), 2 * results_removing_stopwords['softcosine'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['euclidean'].mean(), 2 * results_removing_stopwords['euclidean'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['wmd'].mean(), 2 * results_removing_stopwords['wmd'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['relax-wmd'].mean(), 2 * results_removing_stopwords['relax-wmd'].std()),
                             ])

    #display(pd.DataFrame(table, columns=['Experiment', 'RDN-B'])) #, 'Transfer Learning with Revision Theory - Relax WMD'])) #, 'Learning from scratch (RDN-B)']))
    display(pd.DataFrame(table, columns=['Experiment','RDN-B', 'TreeBoostler', 'Transfer Learning - SoftCosine', 'Transfer Learning - Euclidean', 'Transfer Learning - WMD', 'Transfer Learning - Relax-WMD'])) #, 'Learning from scratch (RDN-B)']))
    #display(pd.DataFrame(table, columns=['Experiment', 'Transfer Learning', 'Transfer Learning with Revision Theory']))

# Results for CLL

,Experiment,RDN-B,TreeBoostler,Transfer Learning - SoftCosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - Relax-WMD
0,9_nell_sports_nell_finances,-0.303 +/- 0.006,-0.307 +/- 0.003,-0.366 +/- 0.001,-0.365 +/- 0.002,-0.365 +/- 0.001,-0.368 +/- 0.003
1,10_nell_finances_nell_sports,-0.085 +/- 0.001,NaN +/- NaN,-0.373 +/- 0.000,-0.344 +/- 0.004,-0.374 +/- 0.008,-0.370 +/- 0.008


# Results for AUC ROC

,Experiment,RDN-B,TreeBoostler,Transfer Learning - SoftCosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - Relax-WMD
0,9_nell_sports_nell_finances,0.762 +/- 0.041,0.746 +/- 0.019,0.558 +/- 0.000,0.562 +/- 0.003,0.540 +/- 0.002,0.541 +/- 0.003
1,10_nell_finances_nell_sports,0.994 +/- 0.001,NaN +/- NaN,0.484 +/- 0.000,0.759 +/- 0.000,0.487 +/- 0.004,0.489 +/- 0.006


# Results for AUC PR

,Experiment,RDN-B,TreeBoostler,Transfer Learning - SoftCosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - Relax-WMD
0,9_nell_sports_nell_finances,0.089 +/- 0.017,0.092 +/- 0.009,0.002 +/- 0.000,0.002 +/- 0.000,0.002 +/- 0.000,0.002 +/- 0.000
1,10_nell_finances_nell_sports,0.313 +/- 0.047,NaN +/- NaN,0.002 +/- 0.000,0.005 +/- 0.000,0.002 +/- 0.000,0.002 +/- 0.000


# Results for Learning time

,Experiment,RDN-B,TreeBoostler,Transfer Learning - SoftCosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - Relax-WMD
0,9_nell_sports_nell_finances,42.075 +/- 9.081,278.228 +/- 97.144,8.447 +/- 0.044,7.439 +/- 0.510,7.442 +/- 0.150,46.458 +/- 0.165
1,10_nell_finances_nell_sports,349.809 +/- 11.872,NaN +/- NaN,28.553 +/- 0.451,20.899 +/- 0.329,17.750 +/- 5.045,59.159 +/- 1.520
